# 🎬 Producción de Contenido Automatizada con IA

Flujo completo: **TXT transcrito** → **LLM analiza** → Edición de video → Multiplicación de contenido

## 🔑 Diferencia clave:
- ❌ **NO transcribe** (tú ya tienes el TXT)
- ✅ **SÍ usa LLM** (Gemini/GPT) para analizar qué segmentos resaltar y cuáles eliminar

**Pasos:**
1. Configurar entorno e instalar dependencias
2. Conectar Google Drive y subir archivos (video + TXT)
3. **PASO 1**: LLM analiza transcripción y sugiere cortes inteligentes
4. **PASO 2**: Editar video (cortes, audio, limpieza)
5. **PASO 3**: Multiplicar contenido (múltiples ratios y plataformas)


## 1️⃣ Instalación y Configuración


In [ ]:
# Instalar dependencias
%pip install -q faster-whisper torch torchaudio moviepy ffmpeg-python pydub


In [ ]:
# Configurar rutas y clonar/actualizar repo
import os
import sys
from pathlib import Path

# Montar Drive
from google.colab import drive
drive.mount('/content/drive')

# Clonar o actualizar el repo
REPO_URL = "https://github.com/tu-usuario/contenido-automatico.git"  # Cambiar por tu repo
PROJECT_ROOT = Path("/content/contenido-automatico")

if not PROJECT_ROOT.exists():
    print("[*] Clonando repositorio...")
    !git clone {REPO_URL} {PROJECT_ROOT}
else:
    print("[*] Actualizando repositorio...")
    !cd {PROJECT_ROOT} && git pull

# Agregar al path
sys.path.insert(0, str(PROJECT_ROOT))

# Crear carpetas necesarias
(PROJECT_ROOT / "output").mkdir(exist_ok=True)
(PROJECT_ROOT / "samples").mkdir(exist_ok=True)

print(f"✅ Proyecto configurado en: {PROJECT_ROOT}")


In [ ]:
# Verificar GPU
from src.utils import verificar_entorno
info = verificar_entorno()


## 2️⃣ Configurar Archivos de Entrada

Necesitamos dos archivos:
- **Video original** (MP4)
- **Transcripción** (TXT con formato timestamp)


In [ ]:
# Configurar rutas de archivos
VIDEO_PATH = "/content/drive/MyDrive/videos/mi_video.mp4"  # Cambiar por tu ruta
TRANSCRIPCION_PATH = "/content/drive/MyDrive/videos/mi_video_transcripcion.txt"  # TXT con transcripción

VIDEO_PATH = Path(VIDEO_PATH)
TRANSCRIPCION_PATH = Path(TRANSCRIPCION_PATH)

print(f"📹 Video: {VIDEO_PATH}")
print(f"   Existe: {VIDEO_PATH.exists()}")
print(f"📝 Transcripción: {TRANSCRIPCION_PATH}")
print(f"   Existe: {TRANSCRIPCION_PATH.exists()}")


---
# PASO 1: Análisis Inteligente de Transcripción

Lee la transcripción y usa un LLM para identificar segmentos clave.


### 1.1 Leer Transcripción


In [ ]:
# Leer archivo de transcripción
print("[*] Leyendo transcripción...")
with open(TRANSCRIPCION_PATH, 'r', encoding='utf-8') as f:
    transcripcion_raw = f.read()

print(f"✅ Transcripción cargada")
print(f"   Tamaño: {len(transcripcion_raw)} caracteres")
print(f"\n📝 Primeras líneas:")
print(transcripcion_raw[:300] + "...")


### 1.2 Configurar LLM (Gemini o OpenAI)


In [ ]:
# Instalar SDK de Gemini
%pip install -q google-generativeai

import google.generativeai as genai
from google.colab import userdata

# 🔑 Configurar API key
# IMPORTANTE: Necesitas una API key de Gemini (gratuita)
# Ver guía completa: https://github.com/tu-usuario/contenido-automatico/blob/main/docs/SETUP_GEMINI.md
#
# Pasos rápidos:
# 1. Ve a: https://aistudio.google.com/app/apikey
# 2. Crea una API key
# 3. En Colab, haz clic en 🔑 (Secrets) en la barra lateral
# 4. Agrega: Nombre="GEMINI_API_KEY", Valor=tu_api_key
# 5. Activa el toggle para dar acceso al notebook

try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("✅ LLM configurado: Gemini 1.5 Flash")
except Exception as e:
    print("❌ Error al configurar Gemini")
    print(f"   {e}")
    print("\n💡 Solución:")
    print("   1. Ve a: https://aistudio.google.com/app/apikey")
    print("   2. Crea una API key")
    print("   3. En Colab: 🔑 Secrets → Add GEMINI_API_KEY")
    print("   4. Vuelve a ejecutar esta celda")


### 1.3 Analizar con LLM: Identificar Segmentos Clave


In [ ]:
prompt_analisis = f"""
Eres un experto editor de video. Analiza esta transcripción de una videollamada y dame:

1. **Momentos destacados**: Identifica los segmentos MÁS interesantes, útiles o con mejor contenido
2. **Momentos a eliminar**: Silencios largos, saludos iniciales, despedidas, conversaciones irrelevantes
3. **Timestamps sugeridos**: Dame los rangos de tiempo exactos en formato MM:SS (minutos:segundos)

**Criterios de selección:**
- ✅ Contenido educativo, insights valiosos, discusiones profundas, decisiones importantes
- ✅ Explicaciones claras de conceptos, debates constructivos
- ❌ Evitar: charlas casuales, logística, problemas técnicos, silencios, saludos/despedidas
- Busca segmentos de 1 a 5 minutos

**Formato de respuesta (IMPORTANTE - usa EXACTAMENTE este formato):**
```
SEGMENTO 1: MM:SS - MM:SS
Razón: [explicación breve]

SEGMENTO 2: MM:SS - MM:SS
Razón: [explicación breve]

SEGMENTO 3: MM:SS - MM:SS
Razón: [explicación breve]
```

TRANSCRIPCIÓN:
{transcripcion_raw}
"""

print("[*] Analizando con Gemini (esto puede tomar 10-30 segundos)...")
response = model.generate_content(prompt_analisis)
analisis_llm = response.text

print("\n" + "="*80)
print("🤖 ANÁLISIS DEL LLM - SEGMENTOS RECOMENDADOS")
print("="*80)
print(analisis_llm)
print("\n" + "="*80)


### 1.3.5 (Opcional) Segundo Análisis: Extracción Automática de Timestamps

Si el LLM generó los timestamps correctamente, podemos extraerlos automáticamente.


In [ ]:
import re

def extraer_timestamps_automaticos(texto_llm):
    """
    Intenta extraer automáticamente los timestamps del análisis del LLM.
    Busca patrones como "MM:SS - MM:SS" o "HH:MM:SS - HH:MM:SS"
    """
    # Patrón regex para timestamps: MM:SS o HH:MM:SS
    patron = r'(\d{1,2}:\d{2}(?::\d{2})?)\s*-\s*(\d{1,2}:\d{2}(?::\d{2})?)'
    matches = re.findall(patron, texto_llm)
    
    def timestamp_to_seconds(ts):
        parts = ts.split(':')
        if len(parts) == 2:  # MM:SS
            return int(parts[0]) * 60 + int(parts[1])
        elif len(parts) == 3:  # HH:MM:SS
            return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])
        return 0
    
    cortes = []
    for inicio_str, fin_str in matches:
        inicio = timestamp_to_seconds(inicio_str)
        fin = timestamp_to_seconds(fin_str)
        if fin > inicio:  # Validar que tenga sentido
            cortes.append((inicio, fin))
    
    return cortes

# Intentar extracción automática
cortes_auto = extraer_timestamps_automaticos(analisis_llm)

if cortes_auto:
    print("✅ Timestamps extraídos automáticamente:")
    print("\nCortes encontrados:")
    print("CORTES_INTELIGENTES = [")
    for inicio, fin in cortes_auto:
        print(f"    ({inicio}, {fin}),  # {inicio//60:02d}:{inicio%60:02d} - {fin//60:02d}:{fin%60:02d}")
    print("]")
    print("\n💡 Copia este código a la siguiente celda si te parece correcto")
else:
    print("⚠️ No se pudieron extraer timestamps automáticamente")
    print("   Revisa el formato del LLM y convierte manualmente en la siguiente celda")


### 1.4 Definir Cortes Finales

Opción A: Si la extracción automática funcionó, copia el código de arriba  
Opción B: Convierte manualmente los timestamps del LLM a segundos

**Formato:** `(inicio_segundos, fin_segundos)`


In [ ]:
# Opción A: Usar los cortes automáticos si salieron bien
# CORTES_INTELIGENTES = cortes_auto

# Opción B: Definir manualmente (recomendado para mayor control)
# Ejemplo basado en análisis del LLM:
CORTES_INTELIGENTES = [
    (163, 524),   # 02:43 - 08:44 - Discusión sobre fundación vs empresa
    (847, 1320),  # 14:07 - 22:00 - Explicación del contexto de PRESS
    (1694, 2282), # 28:14 - 38:02 - Propuesta de pensamiento computacional
]

print("✂️ SHOTLIST FINAL (Basada en análisis LLM)")
print("="*80)
for idx, (inicio, fin) in enumerate(CORTES_INTELIGENTES, 1):
    duracion = fin - inicio
    print(f"Clip {idx}: {inicio}s ({inicio//60:02d}:{inicio%60:02d}) - {fin}s ({fin//60:02d}:{fin%60:02d})")
    print(f"  Duración: {duracion}s ({duracion//60}min {duracion%60}s)")
    print()


📋 **CHECKPOINT: Revisar Antes de Continuar**

Antes de proceder con la edición:
1. ✅ **Revisa los segmentos elegidos** - ¿Tienen sentido según el análisis?
2. ✅ **Verifica los timestamps** - ¿Están en el rango correcto?
3. ✅ **Confirma que el video existe** - ¿La ruta VIDEO_PATH es correcta?

Si todo se ve bien, continúa con el Paso 2 (Edición de Video)


---
# PASO 2: Edición de Video

Cortar clips, limpiar audio, normalizar volumen, exportar.


### 2.1 Cortar y Concatenar Clips


In [ ]:
from src.edicion import cortar_video, concatenar_clips

# Cortar cada clip según los timestamps inteligentes del LLM
clips_paths = []
output_clips_dir = PROJECT_ROOT / "output" / "clips"
output_clips_dir.mkdir(exist_ok=True)

print("✂️ Cortando clips según análisis del LLM...")
for idx, (inicio, fin) in enumerate(CORTES_INTELIGENTES, 1):
    clip_path = output_clips_dir / f"clip_{idx:02d}.mp4"
    print(f"  Clip {idx}: {inicio}s - {fin}s")
    
    cortar_video(
        VIDEO_PATH,
        inicio=inicio,
        fin=fin,
        output_path=clip_path
    )
    clips_paths.append(clip_path)

print(f"\n✅ {len(clips_paths)} clips cortados")

# Concatenar si hay múltiples clips
if len(clips_paths) > 1:
    print("\n[*] Concatenando clips...")
    video_editado = PROJECT_ROOT / "output" / "video_editado.mp4"
    concatenar_clips(clips_paths, video_editado)
    print(f"✅ Video editado: {video_editado}")
else:
    video_editado = clips_paths[0]
    print(f"✅ Video editado (clip único): {video_editado}")


### 2.2 Limpieza de Audio


In [ ]:
from src.edicion import reducir_ruido_basico

print("[*] Reduciendo ruido de audio...")
video_limpio = PROJECT_ROOT / "output" / "video_limpio.mp4"
reducir_ruido_basico(video_editado, video_limpio)
print(f"✅ Audio limpio: {video_limpio}")


### 2.3 Normalizar Audio


In [ ]:
from src.edicion import normalizar_audio_video

print("[*] Normalizando loudness a -14 LUFS...")
video_master = PROJECT_ROOT / "output" / "video_master.mp4"
normalizar_audio_video(video_limpio, video_master, target_lufs=-14)
print(f"✅ Video master listo: {video_master}")


### 2.4 Preview del Video Master


In [ ]:
from IPython.display import Video as VideoPlayer

print("🎬 VIDEO MASTER:")
VideoPlayer(str(video_master), width=800)


---
# PASO 3: Multiplicación de Contenido

Generar variantes para diferentes plataformas.


### 3.1 Exportar Múltiples Ratios


In [ ]:
from src.edicion import export_multi_ratio

# Definir qué ratios generar
RATIOS = ['16:9', '9:16', '1:1']  # YouTube, TikTok/Reels, Instagram

output_variantes = PROJECT_ROOT / "output" / "variantes"

print("[*] Generando variantes de aspect ratio...")
variantes = export_multi_ratio(video_master, output_variantes, ratios=RATIOS)

print("\n✅ VARIANTES GENERADAS:")
for ratio, path in variantes.items():
    print(f"  {ratio}: {path}")


### 3.2 Descargar Resultados


In [ ]:
# Opción 1: Copiar a Google Drive
import shutil

drive_output = Path("/content/drive/MyDrive/contenido_output")
drive_output.mkdir(exist_ok=True)

print("[*] Copiando resultados a Google Drive...")
shutil.copy2(video_master, drive_output / video_master.name)

for ratio, path in variantes.items():
    shutil.copy2(path, drive_output / path.name)

print(f"\n✅ Resultados copiados a: {drive_output}")


In [ ]:
# Opción 2: Descargar directo desde Colab
from google.colab import files

# Descomentar las líneas que quieras descargar
# files.download(str(video_master))
# for ratio, path in variantes.items():
#     files.download(str(path))


---
## ✅ Proceso Completo

Has completado:
1. ✅ Análisis de transcripción
2. ✅ Creación de shotlist
3. ✅ Edición de video (cortes, limpieza, normalización)
4. ✅ Multiplicación de contenido (múltiples ratios)

Los archivos están listos para:
- 🎬 YouTube (16:9)
- 📱 TikTok/Reels (9:16)
- 📸 Instagram (1:1)

**Próximos pasos manuales:**
- Agregar subtítulos (hardcoded o softcoded)
- Agregar overlays y gráficos
- Thumbnail y metadata
- Publicar en plataformas
